Gender dependent training
---------------------------

Gender loss를 붙여서 각 gender 안에서 잘 구분할 수 있는 feature를 만들 수 있을까?

no additional network

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append('../../sv_system/')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

### Configuration

In [3]:
from utils.parser import set_train_config
import easydict
args = easydict.EasyDict(dict(dataset="voxc1_mfcc30", data_folder="/dataset/SV_sets/voxceleb12/feats/mfcc30/",
                              input_frames=800, splice_frames=[200, 800], stride_frames=1, input_format='mfcc',
                              input_dim=30,
                              cuda=True,
                              lrs=[0.1, 0.01], lr_schedule=[20], seed=1337,
                              no_eer=True,
                              batch_size=256, num_workers=8,
                              arch="tdnn_xvector", loss="softmax",
                              n_epochs=50, n_labels=1211,
                             ))
config = set_train_config(args)

### Dataset and Dataloader

In [4]:
voxc1_si_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_si_train_dataframe.pkl")
voxc1_sv_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_sv_test_dataframe.pkl")

si_spks = voxc1_si_df.spk.unique().tolist()

In [5]:
import torch.utils.data as data
from collections  import OrderedDict

class MTLfeatDataset(data.Dataset):
    def __init__(self, data, set_type, config):
        super().__init__()
        self.set_type = set_type
        # data
        self.files = list(data.keys())
        self.labels = list(data.values())
        self.data_folder = config["data_folder"]
        # input audio config
        self.input_frames = config["input_frames"]
        self.input_clip = config["input_clip"]
        self.input_dim = config["input_dim"]

        if set_type == "train":
            self.random_clip = False
        else:
            self.random_clip = False

    def preprocess(self, example):
        # file_data = self._file_cache.get(example)
        data = np.load(example)
        # self._file_cache[example] = data

        # clipping
        in_len = self.input_frames
        if self.input_clip:
            if len(data) > in_len:
                if self.random_clip:
                    start_sample = np.random.randint(0, len(data) - in_len)
                else:
                    start_sample = 0
                data = data[start_sample:start_sample+in_len]
            else:
                gap = max(0, in_len - len(data))

                # # zero-padding
                # data = np.pad(data, (0, gap), "constant")

                # repeat, it shows better result
                repeat = int(np.floor(gap / len(data)))
                residual = gap % len(data)
                # print(f"in_len: {in_len}, data: {len(data)}, repeat: {repeat}, residual: {residual}")
                data = np.concatenate([np.tile(data, (repeat+1, 1)), data[:residual]], axis=0)

        #TODO why do they have diffrent input dimension?
        data = data[:,:self.input_dim] # first dimension could be energy term
        # expand a singleton dimension standing for a channel dimension
        data = torch.from_numpy(data).unsqueeze(0).float()
        return data

    @classmethod
    def read_df(cls, config, df, set_type):
        files = df.feat.tolist()
        if "label" in df.columns:
            labels = df.label.tolist()
        else:
            labels = [-1] * len(df)

        gender = df.gender.apply(lambda x: 1 if x=='m' else 0).values
        samples = OrderedDict(zip(files, zip(labels, gender)))
        dataset = cls(samples, set_type, config)
        return dataset

    def __getitem__(self, index):
        return self.preprocess(os.path.join(self.data_folder, self.files[index])), self.labels[index]

    def __len__(self):
        return len(self.labels)

In [6]:
from data.data_utils import find_dataset, find_trial

trial = find_trial(config, basedir='../../')

=> loaded trial: voxc12_test_trial


In [7]:
train_dataset = MTLfeatDataset.read_df(config, voxc1_si_df[voxc1_si_df.set == 'train'], "train")
val_dataset = MTLfeatDataset.read_df(config, voxc1_si_df[voxc1_si_df.set == 'val'], "val")

In [8]:
def _mtl_collate_fn(batch):
    """
    collate_fn for multi labels
    """
    data, label = list(zip(*batch))
    data = torch.cat(data)
    y1, y2 = list(zip(*label))
    
    return data, torch.LongTensor(y1), torch.LongTensor(y2)

In [9]:
import torch
from torch.utils.data.dataloader import default_collate
batch_size = config["batch_size"]
num_workers = config["num_workers"]

train_loader = torch.utils.data.DataLoader(train_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        pin_memory=False,
        collate_fn=_mtl_collate_fn)

val_loader = torch.utils.data.DataLoader(val_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False,
        pin_memory=False,
        collate_fn=_mtl_collate_fn)

### Define Model

In [10]:
import torch.nn as nn
from model.tdnnModel import st_pool_layer

class tdnn_xvector(nn.Module):
    """xvector architecture"""
    def __init__(self, config, n_spk_labels, n_gender_labels):
        super(tdnn_xvector, self).__init__()
        
        inDim = config['input_dim']
        self.extractor = nn.Sequential(
            nn.Conv1d(inDim, 512, stride=1, dilation=1, kernel_size=5),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Conv1d(512, 512, stride=1, dilation=3, kernel_size=3),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Conv1d(512, 512, stride=1, dilation=4, kernel_size=3),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Conv1d(512, 512, stride=1, dilation=1, kernel_size=1),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Conv1d(512, 1500, stride=1, dilation=1, kernel_size=1),
            nn.BatchNorm1d(1500),
            nn.ReLU(True),
            st_pool_layer(),
            nn.Linear(3000, 512),
        )

        self.spk_classifier = nn.Sequential(
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Linear(512, n_spk_labels)
        )
        
        self.gender_classifier = nn.Sequential(
            nn.Linear(512, 2)
        )

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.Conv1d):
                n = m.kernel_size[0] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()
                
    def embed(self, x):
        x = x.squeeze(1)
        # (batch, time, freq) -> (batch, freq, time)
        x = x.permute(0,2,1)
        x = self.extractor(x)

        return x
    

    def forward(self, x):
        x_ = self.embed(x)
        x1 = self.spk_classifier(x_)
        x2 = self.gender_classifier(x_)

        return x1, x2


### Model Train

In [23]:
model = tdnn_xvector(config,  len(si_spks), 2)

In [24]:
if not config["no_cuda"]:
    model.cuda()
else:
    model = model.cpu()

In [25]:
import torch
from train.train_utils import set_seed, find_optimizer
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion, optimizer = find_optimizer(config, model)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5)

In [26]:
set_seed(config)

In [27]:
import torch
from train.si_train import val
from eval.sv_test import sv_test

config['lamb'] = 0.5
print("lamb: {}".format(config['lamb']))
for epoch_idx in range(0, config['n_epochs']):
    print("-"*30)
    curr_lr = optimizer.state_dict()['param_groups'][0]['lr']
    print("curr_lr: {}".format(curr_lr))

#==============================train code==============================
    model.train()
    loss_sum = 0
    loss_spk_sum = 0
    loss_gender_sum = 0
    corrects = 0
    total = 0
    lamb = config['lamb']
    print_steps = (np.arange(0, 1, 0.1) \
                    * len(train_loader)).astype(np.int64)

    splice_frames = config['splice_frames']
    if len(splice_frames) > 1:
        splice_frames_ = np.random.randint(splice_frames[0], splice_frames[1])
    else:
        splice_frames_ = splice_frames[-1]

    for batch_idx, (X, y_spk, y_gender) in enumerate(train_loader):
        # X.shape is (batch, channel, time, bank)
        X = X.narrow(1, 0, splice_frames_)
        if not config["no_cuda"]:
            X = X.cuda()
            y_spk = y_spk.cuda()
            y_gender = y_gender.cuda()
        optimizer.zero_grad()
        logit_spk, logit_gender = model(X)
        
        loss_spk = criterion(logit_spk, y_spk) 
        loss_gender = criterion(logit_gender, y_gender)
        loss = loss_spk * (1-lamb) + loss_gender  * (lamb)
        
        loss_sum += loss.item()
        loss_spk_sum += loss_spk.item()
        loss_gender_sum += loss_gender.item()
        
        loss.backward()
        optimizer.step()
        
        predicted = torch.argmax(logit_spk, dim=1)
        corrects += predicted.eq(y_spk).cpu().sum().float()
        total += y_spk.size(0)
        
        if batch_idx in print_steps:
            print("train loss: {:.4f}, spk_loss: {:.4f}, gender_loss: {:.4f}, acc: {:.5f} " \
                  .format(loss_sum/total, loss_spk_sum/total, loss_gender_sum/total, corrects/total)
                 )            
    train_acc = corrects/total
    train_loss = loss_sum/total
    print("epoch #{}, train accuracy: {}".format(epoch_idx, train_acc))
    
    scheduler.step(train_loss)
    
#==============================validation code==============================
    total = 0
    corrects = 0
    for batch_idx, (X, y_spk, y_gender) in enumerate(val_loader):
        # X.shape is (batch, channel, time, bank)
        if not config["no_cuda"]:
            X = X.cuda()
            y_spk = y_spk.cuda()
            y_gender = y_gender.cuda()
        optimizer.zero_grad()
        logit_spk, logit_gender = model(X)
        predicted = torch.argmax(logit_spk, dim=1)
        corrects += predicted.eq(y_spk).cpu().sum().float()
        total += y_spk.size(0)
        
    val_acc = corrects/total

    
    print("epoch #{}, val accuracy: {}".format(epoch_idx, val_acc))

#==============================evaluate best_metric==============================
    if not config['no_eer']:
        # eer validation code
        eer, label, score = sv_test(config, sv_loader, model, trial)
        print("epoch #{}, sv eer: {}".format(epoch_idx, eer))
    
    

lamb: 0.5
------------------------------
curr_lr: 0.1
train loss: 0.0152, spk_loss: 0.0278, gender_loss: 0.0027, acc: 0.00000 
train loss: 0.0133, spk_loss: 0.0255, gender_loss: 0.0010, acc: 0.03592 
train loss: 0.0122, spk_loss: 0.0236, gender_loss: 0.0007, acc: 0.06216 
train loss: 0.0114, spk_loss: 0.0221, gender_loss: 0.0006, acc: 0.08898 
train loss: 0.0107, spk_loss: 0.0209, gender_loss: 0.0006, acc: 0.11229 
train loss: 0.0102, spk_loss: 0.0199, gender_loss: 0.0005, acc: 0.13637 
train loss: 0.0097, spk_loss: 0.0190, gender_loss: 0.0005, acc: 0.15876 
train loss: 0.0093, spk_loss: 0.0182, gender_loss: 0.0005, acc: 0.17909 
train loss: 0.0090, spk_loss: 0.0175, gender_loss: 0.0005, acc: 0.19813 
train loss: 0.0087, spk_loss: 0.0169, gender_loss: 0.0004, acc: 0.21774 
epoch #0, train accuracy: 0.2349284589290619
epoch #0, val accuracy: 0.13181228935718536
------------------------------
curr_lr: 0.1
train loss: 0.0050, spk_loss: 0.0095, gender_loss: 0.0004, acc: 0.46094 
train loss

train loss: 0.0002, spk_loss: 0.0004, gender_loss: 0.0001, acc: 0.98415 
train loss: 0.0002, spk_loss: 0.0004, gender_loss: 0.0001, acc: 0.98447 
train loss: 0.0002, spk_loss: 0.0003, gender_loss: 0.0001, acc: 0.98476 
train loss: 0.0002, spk_loss: 0.0003, gender_loss: 0.0001, acc: 0.98487 
epoch #9, train accuracy: 0.9850629568099976
epoch #9, val accuracy: 0.49486374855041504
------------------------------
curr_lr: 0.1
train loss: 0.0001, spk_loss: 0.0001, gender_loss: 0.0001, acc: 1.00000 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0001, acc: 0.99648 
train loss: 0.0001, spk_loss: 0.0001, gender_loss: 0.0001, acc: 0.99740 
train loss: 0.0001, spk_loss: 0.0001, gender_loss: 0.0001, acc: 0.99755 
train loss: 0.0001, spk_loss: 0.0001, gender_loss: 0.0001, acc: 0.99742 
train loss: 0.0001, spk_loss: 0.0001, gender_loss: 0.0001, acc: 0.99753 
train loss: 0.0001, spk_loss: 0.0001, gender_loss: 0.0001, acc: 0.99766 
train loss: 0.0001, spk_loss: 0.0001, gender_loss: 0.0001, acc: 

train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99786 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99782 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99786 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99786 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99798 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99806 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99811 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99819 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99828 
epoch #19, train accuracy: 0.9983326196670532
epoch #19, val accuracy: 0.5397923588752747
------------------------------
curr_lr: 0.010000000000000002
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_l

train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
epoch #28, train accuracy: 1.0
epoch #28, val accuracy: 0.5592020153999329
------------------------------
curr_lr: 0.010000000000000002
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, ac

epoch #37, train accuracy: 1.0
epoch #37, val accuracy: 0.5597966909408569
------------------------------
curr_lr: 0.0010000000000000002
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0001, gender_loss: 0.0000, acc: 1.00000 
epoch #38, train accuracy: 1.0
epoch #38, val accuracy: 0.5595804452896118
------------------------------
curr_lr: 0.0010000000000000

train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0000, spk_loss: 0.0000, gender_loss: 0.0000, acc: 1.00000 
epoch #47, train accuracy: 1.0
epoch #47, val accuracy: 0.5592560768127441
------------------------------
curr_lr: 1.0000000000000004e-05
train loss: 0.0001, spk_loss: 0.0003, gender_loss: 0.0000, acc: 1.00000 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99969 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99946 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, acc: 0.99946 
train loss: 0.0001, spk_loss: 0.0002, gender_loss: 0.0000, 

In [ ]:
torch.save(model.cpu().state_dict(), open("gcommand_ResNet34_v1_mtl_lamb0.3.pt", "wb"))

### SV_Test

equal_sent and diff_sent

In [ ]:
from sv_system.sv_score.score_utils import embeds_utterance

def sv_test(config, sv_loader, model, trial):
    embeddings, _ = embeds_utterance(config, sv_loader, model, lda=None)
    sim_matrix = F.cosine_similarity(
            embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=2)
    cord = [trial.enrolment_id.tolist(), trial.test_id.tolist()]
    score_vector = sim_matrix[cord].numpy()
    label_vector = np.array(trial.label)
    fpr, tpr, thres = roc_curve(
            label_vector, score_vector, pos_label=1)
    eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]

    return eer, label_vector, score_vector

In [ ]:
equal_sent_trial = trial[trial.equal_command]
diff_sent_trial = trial[~trial.equal_command]

In [ ]:
model.cuda()
equal_sent_eer, _, _ = sv_test(config, sv_loader, model, equal_sent_trial)
diff_sent_eer, _, _ = sv_test(config, sv_loader, model, diff_sent_trial)

In [ ]:
# gcommand_ResNet34_v1_mtl_lamb0.1.pt
print(f"equal: {equal_sent_eer}\ndiff: {diff_sent_eer}")

In [ ]:
config['lamb']